In [2]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import scipy
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
dfr = hd.read_raw_data("777")

Time for read_raw_data                 : 5.06 seconds


In [4]:
dft = hd.read_txn_data("777")

Time for read_txn_data                 : 3.25 seconds


In [29]:
dfa = hd.read_analysis_data("XX7")

Time for read_analysis_data            : 2.97 seconds


In [7]:
ag.grocery_shop_entropy(dft)

Time for grocery_shop_entropy          : 0.06 seconds


entropy_groc  entropy_groc_n  entropy_groc_z  entropy_groc_s  \
user_id ym                                                                      
777     2012-02      0.918296        0.276435       -0.300217        3.180833   
        2012-04      0.000000        0.000000       -1.645855        3.277613   
        2012-05      1.918296        0.577465        1.165148        2.971747   
        2012-06      0.918296        0.276435       -0.300217        3.180833   
        2012-07      1.721928        0.518352        0.877398        2.960964   
...                       ...             ...             ...             ...   
587777  2020-03      2.170951        0.653521        1.535379        2.892811   
        2020-04      2.038160        0.613547        1.340792        2.963810   
        2020-05      2.246439        0.676246        1.645998        3.108695   
        2020-06      2.352746        0.708247        1.801776        3.042481   
        2020-07      2.186704        0.658264        1.558464        3.064970   

                 entropy_groc_sn  entropy_groc_sz  
user_id ym                                         
777     2012-02         0.957526         0.993167  
        2012-04         0.986660         1.278977  
        2012-05         0.894585         0.375701  
        2012-06         0.957526         0.993167  
        2012-07         0.891339         0.343856  
...                          ...              ...  
587777  2020-03         0.870823         0.142589  
        2020-04         0.892196         0.352260  
        2020-05         0.935810         0.780131  
        2020-06         0.915878         0.584591  
        2020-07         0.922648         0.651003  

[6217 rows x 6 columns]